<img src="../Img/Python_SQL_PI.png" style="width:100%">


<p style="font-size:22px;text-align:center">Семинар 4.1. Подзапросы</p> 

<p style="font-size:22px;text-align:center">Домашнее задание</p>

<a id = Home></a>

17 марта 2022 года, пн<br>
Поток: ПИ20-1, ПИ20-2, ПИ20-3, ПИ20-4, ПИ20-5, ПИ20-6 (2 подгруппа)<br>


21 марта 2022 года, чт<br>
ПИ20-1, ПИ20-2, ПИ20-3, ПИ20-4, ПИ20-5, ПИ20-6 (3 подгруппа)<br>
ПИ20-1, ПИ20-2, ПИ20-3, ПИ20-4, ПИ20-5, ПИ20-6 (4 подгруппа)<br>

31 марта 2022 года, чт<br>
ПИ20-1, ПИ20-2, ПИ20-3, ПИ20-4, ПИ20-5, ПИ20-6 (1 подгруппа)

<p style='font-size:120%'>1. Выполнить отображение БД Chinook.</p>

<img src="../Img/chinook_ERD.png" style="width:100%">

<p style='font-size:120%'>2. Выполнить запросы с помощью подзапросов.</p>

2.1. С помощью подзапроса составьте запрос, выполняющий вывод информации о счетах клиентов из Бразилии. Результат должен содержать данные из таблицы Invoice: идентификатор счета, идентификатор клиента, дату выставления счета, адрес (BillingAddress), город (BillingCity) и страну (Billing Country) плательщика. Информацию о стране клиента (Brazil) получите в подзапросе из таблицы Customer (столбец Country).

2.2. С помощью подзапроса покажите фамилию, имя, должность сотрудников, подчиняющихся сотруднику Nancy Edwards.

2.3. С помощью подзапроса покажите идентификатор, название, длительность в минутах трэков в жанре *Classical* продолжительностью более 8 минут.

2.4. С помощью подзапроса покажите альбомы, в составе которых более 25 трэков.

2.5. С помощью подзапроса покажите жанры, в составе которых имеются трэки, длящиеся более 18 минут.

2.6. С помощью подзапроса покажите артистов (Artist), исполняющих произведения в жанре *Classical*.

2.7. С помощью подзапроса найдите артистов (Artist), записавших альбомы, в которых более 20 произведений.

2.8. С помощью подзапроса найдите сотрудников, подчиняющихся рукводителю, проживающему с подчинённым в одном городе.

In [31]:
pip install sqlalchemy

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [32]:
from sqlalchemy import MetaData, Table, create_engine, select, and_, or_, not_, desc
from sqlalchemy.sql import func, distinct, exists
import traceback
import pandas as pd

In [33]:
engine = create_engine('sqlite+pysqlite:///../Data/Chinook_Sqlite.sqlite')
metadata = MetaData() # Объект-контейнер, содержит информацию о схеме данных
metadata.reflect(bind=engine)
conn = engine.connect()
metadata.tables.keys()

dict_keys(['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'Track', 'MediaType', 'Playlist', 'PlaylistTrack'])

In [34]:
Album=Table("Album", metadata, autoload=True, autoload_with="engine")
Artist=Table("Artist", metadata, autoload=True, autoload_with="engine")
Customer=Table("Customer", metadata, autoload=True, autoload_with="engine")
Employee=Table("Employee", metadata, autoload=True, autoload_with="engine")
Genre=Table("Genre", metadata, autoload=True, autoload_with="engine")
Invoice=Table("Invoice", metadata, autolad=True, autolad_with="engine")
InvoiceLine=Table("InvoiceLine", metadata, autoload=True, autoload_with="engine")
Track=Table("Track", metadata, autoload=True, autoload_with="engine")
MediaType=Table("MediaType", metadata, autoload=True, autoload_with="engine")
Playlist=Table("Playlist", metadata, autoload=True, autoload_with="engine")
PlaylistTrack=Table("PlaylistTrack", metadata, autoload=True, autoload_with="engine")

2.1. С помощью подзапроса составьте запрос, выполняющий вывод информации о счетах клиентов из Бразилии. Результат должен содержать данные из таблицы Invoice: идентификатор счета, идентификатор клиента, дату выставления счета, адрес (BillingAddress), город (BillingCity) и страну (Billing Country) плательщика. Информацию о стране клиента (Brazil) получите в подзапросе из таблицы Customer (столбец Country).

In [37]:
# 2.1 

subq = select(Invoice.c.InvoiceDate).scalar_subquery()
subq_2 = select(Invoice.c.BillingCountry).scalar_subquery()

s = select(Customer.c.LastName, Customer.c.FirstName, Customer.c.Country,
           Customer.c.CustomerId, subq.label("InvoiceDate"), subq_2.label("BillingCountry"))
s = s.where(Customer.c.Country == 'Brazil')

print(s)
pd.read_sql(s, conn)

SELECT "Customer"."LastName", "Customer"."FirstName", "Customer"."Country", "Customer"."CustomerId", (SELECT "Invoice"."InvoiceDate" 
FROM "Invoice") AS "InvoiceDate", (SELECT "Invoice"."BillingCountry" 
FROM "Invoice") AS "BillingCountry" 
FROM "Customer" 
WHERE "Customer"."Country" = :Country_1


,LastName,FirstName,Country,CustomerId,InvoiceDate,BillingCountry
0,Gonçalves,Luís,Brazil,1,2009-01-01,Germany
1,Martins,Eduardo,Brazil,10,2009-01-01,Germany
2,Rocha,Alexandre,Brazil,11,2009-01-01,Germany
3,Almeida,Roberto,Brazil,12,2009-01-01,Germany
4,Ramos,Fernanda,Brazil,13,2009-01-01,Germany


2.2. С помощью подзапроса покажите фамилию, имя, должность сотрудников, подчиняющихся сотруднику Nancy Edwards.

In [38]:
# 2.2

subq = select(Employee.c.EmployeeId)
subq = subq.where(Employee.c.LastName == "Edwards").filter(and_(Employee.c.FirstName == "Nancy")).scalar_subquery()

s = select(Employee.c.EmployeeId, Employee.c.LastName, Employee.c.Title)
s = s.where(Employee.c.ReportsTo == subq)

print(s)
pd.read_sql(s, conn)

SELECT "Employee"."EmployeeId", "Employee"."LastName", "Employee"."Title" 
FROM "Employee" 
WHERE "Employee"."ReportsTo" = (SELECT "Employee"."EmployeeId" 
FROM "Employee" 
WHERE "Employee"."LastName" = :LastName_1 AND "Employee"."FirstName" = :FirstName_1)


,EmployeeId,LastName,Title
0,3,Peacock,Sales Support Agent
1,4,Park,Sales Support Agent
2,5,Johnson,Sales Support Agent


2.3. С помощью подзапроса покажите идентификатор, название, длительность в минутах трэков в жанре *Classical* продолжительностью более 8 минут.

In [39]:
# 2.3

subq = select(Genre.c.GenreId).where(Genre.c.Name == "Classical").scalar_subquery()

s = select(Track)
s = s.where(Track.c.GenreId == subq).filter(and_(Track.c.Milliseconds > (1000 * 60 * 8)))

print(s)
pd.read_sql(s, conn)


SELECT "Track"."TrackId", "Track"."Name", "Track"."AlbumId", "Track"."MediaTypeId", "Track"."GenreId", "Track"."Composer", "Track"."Milliseconds", "Track"."Bytes", "Track"."UnitPrice" 
FROM "Track" 
WHERE "Track"."GenreId" = (SELECT "Genre"."GenreId" 
FROM "Genre" 
WHERE "Genre"."Name" = :Name_1) AND "Track"."Milliseconds" > :Milliseconds_1


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/io/sql.py:1402: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  return self.connectable.execution_options().execute(*args, **kwargs)


,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,3404,"Miserere mei, Deus",273,2,24,Gregorio Allegri,501503,8285941,0.99
1,3410,"The Messiah: Behold, I Tell You a Mystery... T...",279,2,24,George Frideric Handel,582029,9553140,0.99
2,3423,"Jupiter, the Bringer of Jollity",292,2,24,Gustav Holst,522099,8547876,0.99
3,3425,"Adagio for Strings from the String Quartet, Op...",294,2,24,Samuel Barber,596519,9585597,0.99
4,3432,"Scheherazade, Op. 35: I. The Sea and Sindbad's...",299,2,24,Nikolai Rimsky-Korsakov,545203,8916313,0.99
5,3434,"Concerto for Piano No. 2 in F Minor, Op. 21: I...",301,2,24,Frédéric Chopin,560342,9160082,0.99
6,3440,"Concerto for Cello and Orchestra in E minor, O...",306,2,24,Edward Elgar,483133,7865479,0.99
7,3445,On the Beautiful Blue Danube,311,2,24,Johann Strauss II,526696,8610225,0.99
8,3446,"Symphonie Fantastique, Op. 14: V. Songe d'une ...",312,2,24,Hector Berlioz,561967,9173344,0.99
9,3485,Symphony No. 3 Op. 36 for Orchestra and Sopran...,330,2,24,Henryk Górecki,567494,9273123,0.99


2.4. С помощью подзапроса покажите альбомы, в составе которых более 25 трэков.

In [40]:
# 2.4
subq = select(func.count(Track.c.TrackId)).where(Track.c.AlbumId == Album.c.AlbumId).label("CountTrack")

s = select(Album, subq).where(subq > 25).order_by(desc(subq))

print(s)
pd.read_sql(s, conn)

SELECT "Album"."AlbumId", "Album"."Title", "Album"."ArtistId", (SELECT count("Track"."TrackId") AS count_1 
FROM "Track" 
WHERE "Track"."AlbumId" = "Album"."AlbumId") AS "CountTrack" 
FROM "Album" 
WHERE (SELECT count("Track"."TrackId") AS count_1 
FROM "Track" 
WHERE "Track"."AlbumId" = "Album"."AlbumId") > :param_1 ORDER BY "CountTrack" DESC


,AlbumId,Title,ArtistId,CountTrack
0,141,Greatest Hits,100,57
1,23,Minha Historia,17,34
2,73,Unplugged,81,30
3,229,"Lost, Season 3",149,26


2.5. С помощью подзапроса покажите жанры, в составе которых имеются трэки, длящиеся более 18 минут.

In [41]:
subq = select(Track.c.GenreId).where(Track.c.Milliseconds > (1000 * 60 * 18))

s = select(Genre).where(Genre.c.GenreId.in_(subq))

print(s)
pd.read_sql(s, conn)

SELECT "Genre"."GenreId", "Genre"."Name" 
FROM "Genre" 
WHERE "Genre"."GenreId" IN (SELECT "Track"."GenreId" 
FROM "Track" 
WHERE "Track"."Milliseconds" > :Milliseconds_1)


,GenreId,Name
0,1,Rock
1,18,Science Fiction
2,19,TV Shows
3,20,Sci Fi & Fantasy
4,21,Drama
5,22,Comedy


2.6. С помощью подзапроса покажите артистов (Artist), исполняющих произведения в жанре *Classical*.

In [42]:
# 2.6
subq = select(Track).join(Genre, Genre.c.GenreId == Track.c.GenreId)
subq = subq.where(Genre.c.Name == "Classical")

s = select(Track.c.Composer)
s = s.where(Track.c.Composer != "Anonymous", Track.c.Composer != "<null>")

print(s)
pd.read_sql(s, conn)

SELECT "Track"."Composer" 
FROM "Track" 
WHERE "Track"."Composer" != :Composer_1 AND "Track"."Composer" != :Composer_2


,Composer
0,"Angus Young, Malcolm Young, Brian Johnson"
1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho..."
2,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D..."
3,Deaffy & R.A. Smith-Diesel
4,"Angus Young, Malcolm Young, Brian Johnson"
...,...
2519,Pietro Antonio Locatelli
2520,Franz Schubert
2521,Claudio Monteverdi
2522,Wolfgang Amadeus Mozart


2.7. С помощью подзапроса найдите артистов (Artist), записавших альбомы, в которых более 20 произведений.

In [43]:
# 2.7 через join
s = select(Artist.c.Name, func.count(Track.c.AlbumId).label("Number of tracks"))
s = s.where(Artist.c.ArtistId == Album.c.ArtistId)
s = s.join(Album, Album.c.AlbumId == Track.c.AlbumId, isouter=True)
s = s.having(func.count(Track.c.AlbumId) > 20)
s = s.group_by(Album.c.AlbumId)

print(s)
pd.read_sql(s, conn)

SELECT "Artist"."Name", count("Track"."AlbumId") AS "Number of tracks" 
FROM "Artist", "Track" LEFT OUTER JOIN "Album" ON "Album"."AlbumId" = "Track"."AlbumId" 
WHERE "Artist"."ArtistId" = "Album"."ArtistId" GROUP BY "Album"."AlbumId" 
HAVING count("Track"."AlbumId") > :count_1


,Name,Number of tracks
0,Chico Buarque,34
1,Chico Science & Nação Zumbi,23
2,Green Day,21
3,Gene Krupa,22
4,Eric Clapton,30
5,Frank Sinatra,24
6,Lenny Kravitz,57
7,Os Paralamas Do Sucesso,21
8,Titãs,22
9,Heroes,23


2.8. С помощью подзапроса найдите сотрудников, подчиняющихся рукводителю, проживающему с подчинённым в одном городе.

In [44]:
'''
SELECT e.last_name "employee", e.city "сity", m.last_name "manager"
FROM employee e JOIN employee m ON m.employee_id = e.reports_to
WHERE e.city = m.city
'''


In [45]:
# 2.8 через join
e = Employee.alias('EMP')
m = Employee.alias('MAN')

j = e.join(m, e.c.ReportsTo == m.c.EmployeeId)
s = select(e.c.LastName.label('Employee'), m.c.LastName.label('Manager'), m.c.City).select_from(j)
s = s.where(e.c.City == m.c.City)

print(s)
pd.read_sql(s, conn)

SELECT "EMP"."LastName" AS "Employee", "MAN"."LastName" AS "Manager", "MAN"."City" 
FROM "Employee" AS "EMP" JOIN "Employee" AS "MAN" ON "EMP"."ReportsTo" = "MAN"."EmployeeId" 
WHERE "EMP"."City" = "MAN"."City"


,Employee,Manager,City
0,Peacock,Edwards,Calgary
1,Park,Edwards,Calgary
2,Johnson,Edwards,Calgary
